In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.1.0


In [2]:
reviews = pd.read_csv("Reviews.csv")

In [3]:
reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
reviews = reviews.dropna()
reviews = reviews.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator',
                        'Score','Time'], 1)
reviews = reviews.reset_index(drop=True)

In [5]:
reviews.head()

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
# A list of contractions from 
# http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [7]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [8]:
word_dict = {}

with open("WordNet/dict/adj.exc") as f:
    content = f.readlines()
    content = [x.strip() for x in content]

for replace in content:
    replace = replace.split()
    word_dict[replace[0]] = replace[1]
    
with open("WordNet/dict/adv.exc") as f:
    content = f.readlines()
    content = [x.strip() for x in content]

for replace in content:
    replace = replace.split()
    word_dict[replace[0]] = replace[1]

with open("WordNet/dict/noun.exc") as f:
    content = f.readlines()
    content = [x.strip() for x in content]

for replace in content:
    replace = replace.split()
    word_dict[replace[0]] = replace[1]

with open("WordNet/dict/verb.exc") as f:
    content = f.readlines()
    content = [x.strip() for x in content]

for replace in content:
    replace = replace.split()
    word_dict[replace[0]] = replace[1]

In [9]:
# Clean the summaries and texts
clean_summaries = []
for summary in reviews.Summary:
    summary = clean_text(summary, remove_stopwords=False)
    
    summary_split = summary.split()
    for i in range(len(summary_split)):
        if summary_split[i] in word_dict.keys():
            summary_split[i] = word_dict[summary_split[i]]
            
    summary = " ".join(summary_split)
    clean_summaries.append(summary)
print("Summaries are complete.")

Summaries are complete.


In [10]:
clean_texts = []
for text in reviews.Text:
    text = clean_text(text, remove_stopwords=True)
    
    text_split = text.split()
    for i in range(len(text_split)):
        if text_split[i] in word_dict.keys():
            text_split[i] = word_dict[text_split[i]]
            
    text = " ".join(text_split)
    clean_texts.append(text)
print("Texts are complete.")

Texts are complete.


In [11]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [12]:
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 131476


In [13]:
# Load Conceptnet Numberbatch's (CN) embeddings, similar to GloVe, but probably better 
# (https://github.com/commonsense/conceptnet-numberbatch)
embeddings_index = {}
with open('numberbatch-en-17.06.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 417195


In [14]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 3843
Percent of words that are missing from vocabulary: 2.92%


In [15]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 131476
Number of words we will use: 58353
Percent of words we will use: 44.379999999999995%


In [16]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

58353


In [17]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [18]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 25679933
Total number of UNKs in headlines: 191682
Percent of words that are UNK: 0.75%


In [19]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [20]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
              counts
count  568411.000000
mean        4.181624
std         2.657872
min         0.000000
25%         2.000000
50%         4.000000
75%         5.000000
max        48.000000

Texts:
              counts
count  568411.000000
mean       41.996835
std        42.520873
min         1.000000
25%        18.000000
50%        29.000000
75%        50.000000
max      2085.000000


In [21]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [22]:
# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 84
max_summary_length = 13
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

425683
425683


**Building the Model**

In [23]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [24]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [25]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [26]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits

In [27]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits

In [28]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
    initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(enc_state[0],
                                                                    _zero_state_tensors(rnn_size, 
                                                                                        batch_size, 
                                                                                        tf.float32)) 
    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)

    return training_logits, inference_logits

In [29]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [30]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [31]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [32]:
# Set the Hyperparameters
epochs = 100
batch_size = 20
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

In [33]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

Graph is built.


In [34]:
# Subset the data for training
start = 50000
end = start + 150000
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 13
The longest text length: 25


In [35]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = "best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Epoch   1/100 Batch   20/7500 - Loss:  4.972, Seconds: 9.28
Epoch   1/100 Batch   40/7500 - Loss:  3.389, Seconds: 9.29
Epoch   1/100 Batch   60/7500 - Loss:  3.325, Seconds: 10.18
Epoch   1/100 Batch   80/7500 - Loss:  3.273, Seconds: 13.18
Epoch   1/100 Batch  100/7500 - Loss:  3.157, Seconds: 10.15
Epoch   1/100 Batch  120/7500 - Loss:  3.159, Seconds: 11.18
Epoch   1/100 Batch  140/7500 - Loss:  3.221, Seconds: 8.45
Epoch   1/100 Batch  160/7500 - Loss:  3.457, Seconds: 12.28
Epoch   1/100 Batch  180/7500 - Loss:  3.289, Seconds: 13.22
Epoch   1/100 Batch  200/7500 - Loss:  3.212, Seconds: 11.28
Epoch   1/100 Batch  220/7500 - Loss:  3.106, Seconds: 10.39
Epoch   1/100 Batch  240/7500 - Loss:  2.983, Seconds: 9.40
Epoch   1/100 Batch  260/7500 - Loss:  3.174, Seconds: 10.28
Epoch   1/100 Batch  280/7500 - Loss:  3.231, Seconds: 12.17
Epoch   1/100 Batch  300/7500 - Loss:  3.073, Seconds: 10.28
Epoch   1/100 Batch  320/7500 - Loss:  3.019, Seconds: 8.54
Epoch   1/100 Batch  340/7500

Epoch   1/100 Batch 2720/7500 - Loss:  2.494, Seconds: 8.66
Epoch   1/100 Batch 2740/7500 - Loss:  2.349, Seconds: 11.51
Epoch   1/100 Batch 2760/7500 - Loss:  2.533, Seconds: 9.74
Epoch   1/100 Batch 2780/7500 - Loss:  2.436, Seconds: 10.68
Epoch   1/100 Batch 2800/7500 - Loss:  2.206, Seconds: 8.84
Epoch   1/100 Batch 2820/7500 - Loss:  2.524, Seconds: 9.62
Epoch   1/100 Batch 2840/7500 - Loss:  2.312, Seconds: 9.73
Epoch   1/100 Batch 2860/7500 - Loss:  2.209, Seconds: 12.61
Epoch   1/100 Batch 2880/7500 - Loss:  2.446, Seconds: 10.73
Epoch   1/100 Batch 2900/7500 - Loss:  2.161, Seconds: 10.61
Epoch   1/100 Batch 2920/7500 - Loss:  2.279, Seconds: 10.89
Epoch   1/100 Batch 2940/7500 - Loss:  2.242, Seconds: 7.79
Epoch   1/100 Batch 2960/7500 - Loss:  1.885, Seconds: 15.53
Epoch   1/100 Batch 2980/7500 - Loss:  2.152, Seconds: 12.69
Epoch   1/100 Batch 3000/7500 - Loss:  2.690, Seconds: 12.75
Epoch   1/100 Batch 3020/7500 - Loss:  2.584, Seconds: 9.89
Epoch   1/100 Batch 3040/7500 -

Epoch   1/100 Batch 5420/7500 - Loss:  1.941, Seconds: 12.04
Epoch   1/100 Batch 5440/7500 - Loss:  2.198, Seconds: 11.10
Epoch   1/100 Batch 5460/7500 - Loss:  2.161, Seconds: 11.95
Epoch   1/100 Batch 5480/7500 - Loss:  2.091, Seconds: 11.98
Epoch   1/100 Batch 5500/7500 - Loss:  2.457, Seconds: 12.09
Epoch   1/100 Batch 5520/7500 - Loss:  2.586, Seconds: 9.19
Epoch   1/100 Batch 5540/7500 - Loss:  2.321, Seconds: 55.84
Epoch   1/100 Batch 5560/7500 - Loss:  2.334, Seconds: 12.13
Epoch   1/100 Batch 5580/7500 - Loss:  2.450, Seconds: 12.14
Epoch   1/100 Batch 5600/7500 - Loss:  2.261, Seconds: 12.13
Epoch   1/100 Batch 5620/7500 - Loss:  2.165, Seconds: 10.10
Epoch   1/100 Batch 5640/7500 - Loss:  2.398, Seconds: 11.04
Epoch   1/100 Batch 5660/7500 - Loss:  2.346, Seconds: 10.42
Epoch   1/100 Batch 5680/7500 - Loss:  2.183, Seconds: 11.99
Epoch   1/100 Batch 5700/7500 - Loss:  2.153, Seconds: 12.16
Epoch   1/100 Batch 5720/7500 - Loss:  2.278, Seconds: 14.15
Epoch   1/100 Batch 5740/

Epoch   2/100 Batch  620/7500 - Loss:  1.801, Seconds: 11.26
Epoch   2/100 Batch  640/7500 - Loss:  1.881, Seconds: 10.39
Epoch   2/100 Batch  660/7500 - Loss:  1.938, Seconds: 13.26
Epoch   2/100 Batch  680/7500 - Loss:  1.866, Seconds: 12.36
Epoch   2/100 Batch  700/7500 - Loss:  1.906, Seconds: 11.35
Epoch   2/100 Batch  720/7500 - Loss:  1.910, Seconds: 9.34
Epoch   2/100 Batch  740/7500 - Loss:  2.109, Seconds: 9.53
Epoch   2/100 Batch  760/7500 - Loss:  1.898, Seconds: 10.31
Epoch   2/100 Batch  780/7500 - Loss:  2.078, Seconds: 14.25
Epoch   2/100 Batch  800/7500 - Loss:  2.056, Seconds: 14.30
Epoch   2/100 Batch  820/7500 - Loss:  1.890, Seconds: 9.44
Epoch   2/100 Batch  840/7500 - Loss:  1.776, Seconds: 8.40
Epoch   2/100 Batch  860/7500 - Loss:  1.972, Seconds: 9.49
Epoch   2/100 Batch  880/7500 - Loss:  2.384, Seconds: 9.49
Epoch   2/100 Batch  900/7500 - Loss:  2.295, Seconds: 8.48
Epoch   2/100 Batch  920/7500 - Loss:  2.298, Seconds: 9.53
Epoch   2/100 Batch  940/7500 - 

Epoch   2/100 Batch 3320/7500 - Loss:  2.443, Seconds: 12.74
Epoch   2/100 Batch 3340/7500 - Loss:  2.151, Seconds: 12.65
Epoch   2/100 Batch 3360/7500 - Loss:  2.465, Seconds: 11.74
Epoch   2/100 Batch 3380/7500 - Loss:  2.380, Seconds: 11.92
Epoch   2/100 Batch 3400/7500 - Loss:  2.077, Seconds: 10.75
Epoch   2/100 Batch 3420/7500 - Loss:  2.216, Seconds: 12.61
Epoch   2/100 Batch 3440/7500 - Loss:  2.098, Seconds: 11.83
Epoch   2/100 Batch 3460/7500 - Loss:  2.316, Seconds: 15.71
Epoch   2/100 Batch 3480/7500 - Loss:  1.928, Seconds: 9.80
Epoch   2/100 Batch 3500/7500 - Loss:  2.276, Seconds: 10.72
Epoch   2/100 Batch 3520/7500 - Loss:  2.339, Seconds: 8.78
Epoch   2/100 Batch 3540/7500 - Loss:  2.276, Seconds: 7.94
Epoch   2/100 Batch 3560/7500 - Loss:  2.343, Seconds: 12.64
Epoch   2/100 Batch 3580/7500 - Loss:  2.090, Seconds: 11.79
Epoch   2/100 Batch 3600/7500 - Loss:  2.114, Seconds: 9.77
Epoch   2/100 Batch 3620/7500 - Loss:  2.183, Seconds: 10.81
Epoch   2/100 Batch 3640/750

...

Epoch   5/100 Batch 2400/7500 - Loss:  1.853, Seconds: 9.59
Epoch   5/100 Batch 2420/7500 - Loss:  1.942, Seconds: 14.44
Epoch   5/100 Batch 2440/7500 - Loss:  1.913, Seconds: 15.53
Epoch   5/100 Batch 2460/7500 - Loss:  1.835, Seconds: 10.65
Epoch   5/100 Batch 2480/7500 - Loss:  1.856, Seconds: 13.73
Average loss for this update: 1.788
New Record!
Epoch   5/100 Batch 2500/7500 - Loss:  1.877, Seconds: 11.64
Epoch   5/100 Batch 2520/7500 - Loss:  1.624, Seconds: 8.74
Epoch   5/100 Batch 2540/7500 - Loss:  1.874, Seconds: 15.67
Epoch   5/100 Batch 2560/7500 - Loss:  1.795, Seconds: 13.69
Epoch   5/100 Batch 2580/7500 - Loss:  1.825, Seconds: 9.82
Epoch   5/100 Batch 2600/7500 - Loss:  1.848, Seconds: 14.49
Epoch   5/100 Batch 2620/7500 - Loss:  1.718, Seconds: 11.68
Epoch   5/100 Batch 2640/7500 - Loss:  1.936, Seconds: 12.62
Epoch   5/100 Batch 2660/7500 - Loss:  1.753, Seconds: 13.63
Epoch   5/100 Batch 2680/7500 - Loss:  1.709, Seconds: 12.50
Epoch   5/100 Batch 2700/7500 - Loss:  1

Epoch   5/100 Batch 5080/7500 - Loss:  1.905, Seconds: 9.23
Epoch   5/100 Batch 5100/7500 - Loss:  2.022, Seconds: 14.96
Epoch   5/100 Batch 5120/7500 - Loss:  1.778, Seconds: 12.15
Epoch   5/100 Batch 5140/7500 - Loss:  1.974, Seconds: 11.96
Epoch   5/100 Batch 5160/7500 - Loss:  1.988, Seconds: 10.99
Epoch   5/100 Batch 5180/7500 - Loss:  1.850, Seconds: 10.05
Epoch   5/100 Batch 5200/7500 - Loss:  2.058, Seconds: 12.92
Epoch   5/100 Batch 5220/7500 - Loss:  1.754, Seconds: 13.03
Epoch   5/100 Batch 5240/7500 - Loss:  1.680, Seconds: 10.00
Epoch   5/100 Batch 5260/7500 - Loss:  1.770, Seconds: 14.82
Epoch   5/100 Batch 5280/7500 - Loss:  1.656, Seconds: 10.98
Epoch   5/100 Batch 5300/7500 - Loss:  1.903, Seconds: 9.11
Epoch   5/100 Batch 5320/7500 - Loss:  1.764, Seconds: 14.01
Epoch   5/100 Batch 5340/7500 - Loss:  1.718, Seconds: 10.03
Epoch   5/100 Batch 5360/7500 - Loss:  1.803, Seconds: 13.05
Epoch   5/100 Batch 5380/7500 - Loss:  1.717, Seconds: 13.04
Epoch   5/100 Batch 5400/7

Epoch   6/100 Batch  280/7500 - Loss:  1.915, Seconds: 12.38
Epoch   6/100 Batch  300/7500 - Loss:  1.866, Seconds: 10.50
Epoch   6/100 Batch  320/7500 - Loss:  1.835, Seconds: 8.42
Epoch   6/100 Batch  340/7500 - Loss:  1.700, Seconds: 9.51
Epoch   6/100 Batch  360/7500 - Loss:  1.781, Seconds: 10.28
Epoch   6/100 Batch  380/7500 - Loss:  1.767, Seconds: 10.32
Epoch   6/100 Batch  400/7500 - Loss:  1.791, Seconds: 8.42
Epoch   6/100 Batch  420/7500 - Loss:  1.687, Seconds: 13.22
Epoch   6/100 Batch  440/7500 - Loss:  1.832, Seconds: 11.35
Epoch   6/100 Batch  460/7500 - Loss:  1.700, Seconds: 11.51
Epoch   6/100 Batch  480/7500 - Loss:  1.805, Seconds: 8.40
Epoch   6/100 Batch  500/7500 - Loss:  1.712, Seconds: 12.30
Epoch   6/100 Batch  520/7500 - Loss:  1.608, Seconds: 7.48
Epoch   6/100 Batch  540/7500 - Loss:  1.755, Seconds: 13.18
Epoch   6/100 Batch  560/7500 - Loss:  1.551, Seconds: 10.30
Epoch   6/100 Batch  580/7500 - Loss:  1.562, Seconds: 9.34
Epoch   6/100 Batch  600/7500 

Epoch   6/100 Batch 2980/7500 - Loss:  1.527, Seconds: 12.52
Epoch   6/100 Batch 3000/7500 - Loss:  2.014, Seconds: 12.60
Epoch   6/100 Batch 3020/7500 - Loss:  1.910, Seconds: 9.69
Epoch   6/100 Batch 3040/7500 - Loss:  1.909, Seconds: 9.84
Epoch   6/100 Batch 3060/7500 - Loss:  1.790, Seconds: 8.79
Epoch   6/100 Batch 3080/7500 - Loss:  1.856, Seconds: 10.30
Epoch   6/100 Batch 3100/7500 - Loss:  1.984, Seconds: 10.77
Epoch   6/100 Batch 3120/7500 - Loss:  1.842, Seconds: 8.74
Epoch   6/100 Batch 3140/7500 - Loss:  1.930, Seconds: 10.70
Epoch   6/100 Batch 3160/7500 - Loss:  1.851, Seconds: 12.60
Epoch   6/100 Batch 3180/7500 - Loss:  1.938, Seconds: 7.87
Epoch   6/100 Batch 3200/7500 - Loss:  1.794, Seconds: 13.63
Epoch   6/100 Batch 3220/7500 - Loss:  1.737, Seconds: 10.75
Epoch   6/100 Batch 3240/7500 - Loss:  1.739, Seconds: 9.79
Epoch   6/100 Batch 3260/7500 - Loss:  1.734, Seconds: 39.84
Epoch   6/100 Batch 3280/7500 - Loss:  1.731, Seconds: 13.61
Epoch   6/100 Batch 3300/7500 

KeyboardInterrupt: 

In [36]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [49]:
# Create your own review or use one from the dataset
#input_sentence = "I have never eaten an apple before, but this red one was nice. \
                  #I think that I will try a green apple next time."
#text = text_to_seq(input_sentence)
random = np.random.randint(0,len(clean_texts))
input_sentence = clean_texts[random]
text = text_to_seq(clean_texts[random])

checkpoint = "./best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(5,8)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove the padding from the tweet
pad = vocab_to_int["<PAD>"] 

print('Original Text:', input_sentence)

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Original Text: jerky strips delicious vegetarian 13 years always looking tasty vegetarian meat replacement foods strips definitely work keep couple car snack time stop eat texture meat like question read ingredients sure make animal flesh buy case texas bbq case hickory smoked fantastic texas bbq nice spicy kick think really enjoy product

Text
  Word Ids:    [1979, 2784, 59, 1870, 2814, 763, 58, 359, 21, 1870, 866, 1934, 479, 2784, 1969, 495, 838, 1744, 2525, 259, 102, 832, 762, 1644, 866, 111, 6057, 589, 469, 453, 318, 8582, 18796, 92, 358, 4658, 642, 358, 3368, 2276, 456, 4658, 642, 15, 241, 262, 1491, 375, 1580, 60]
  Input Words: jerky strips delicious vegetarian 13 years always looking tasty vegetarian meat replacement foods strips definitely work keep couple car snack time stop eat texture meat like question read ingredients sure make animal flesh buy case texas bbq case hickory smoked fantastic texas bbq nice spicy kic

In [45]:
!rm -rf log_simple_graph
!rm -rf log_simple_stat

In [46]:
summary_writer = tf.summary.FileWriter('log_simple_graph', sess.graph)

In [48]:
!tensorboard --logdir=log_simple_graph

Starting TensorBoard b'47' at http://0.0.0.0:6006
(Press CTRL+C to quit)
^C
